# HW 1. Логистическая регрессия

In [1]:
import os

import cv2
import numpy as np
from numpy.typing import NDArray

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## Загрузка данных

Для того чтобы загрузить данные в нейросеть или более простые алгоритмы ML, их необходимо должным образом открыть и преобразовать в вектор с числами. Для этого воспользуемся функцией ```read_files()```.

Этот парсер делает следующие вещи:
 - открывает файл картинки с диска (с помощью библиотеки opencv),
 - проверяет, что картинка действительно открылась и сейчас является матрицей (```np.array```),
 - преобразует матрицу в вектор (путем записи всех столбцов друг под другом),
 - возвращает массив из векторов, в которых хранятся картинки, и лейбл, соответствующий каждой картинке.

In [ ]:
def read_files(path: str, ans: int, target_dim: tuple = (256, 256)) -> dict[str, NDArray[np.float64]]:
    files = os.listdir(path)
    X = None
    for i, name in enumerate(files):
        img = cv2.imread(path + '/' + name, 0) # 0 means black-white picture
        if img.shape != 0:
            img = cv2.resize(img, target_dim, interpolation=cv2.INTER_LINEAR)
            vect = img.reshape(1, 256 ** 2) / 255.

            X = vect if (X is None) else np.vstack((X, vect))
        # print(f"{i}/{len(files)}")
    # print()
    y = np.ones((len(files))) * ans
    return {"data": X, "labels": y}

In [6]:
?np.vstack

Логика программы следующая:
 - есть класс ```LogisticRegression```. Он содержит веса модели и нужен для того, чтобы генерировать предсказания. Также, этот класс снабжен методом backward() для возможности дифференцирования функции потерь по весам этого класса.
 - есть класс ```Loss```. Он определяет функцию потерь, которую мы хотим использовать. Также, этот класс осуществляет подсчет градиентов функции потерь по всем весам модели.
 - есть класс ```Optimizer```. Он отвечает за то, как будут обновляться веса после подсчета градиентов (после работы класса ```Loss```). Сегодня мы рассматриваем простой градиентный спуск, но более сложные модели используют более усовершенствованные алгоритмы оптимизации.

То есть:
 1. Вызывается метод ```forward()``` нашей модели (логистической регрессии). После этого у нас выводятся текущие предсказания модели.
 2. Вызывается метод ```loss()```, который сравнивает предсказания модели с истинными ответами. После этого подсчитываются градиенты функции потерь по всем весам модели.
 3. Подсчитанные градиенты вычитаются из весов (происходит шаг градиентного спуска) путем вызова ```optimizer.step()```

## Логистическая регрессия. Backward propagation
Back propagation реализуется с помощью уравнений, написанных ниже. Для подсчитанной функции потерь:
$$
\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)
$$
где $\hat{y}_i = \sigma(w^T x_i + b)$, где $\sigma$ - сигмоидная функция.

----Здесь необходимо провести расчеты для производных функции потерь по параметрам----

In [ ]:
class LogisticRegression:
    def __init__(self, n_features):
        """Initialize model parameters with zeros."""
        self.w = np.zeros(n_features)
        self.b = 0.0

    def forward(self, X: NDArray[np.float64]) -> NDArray[np.float64]:
        """Compute model predictions a.k.a. forward pass.
        Args:
            X: input data.
        Returns:
            probability predictions.
        """
        z = np.dot(X, self.w) + self.b
        return self.sigmoid(z)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

class Loss:
    def __init__(self):
        pass

    def __call__(self, y_pred, y_true):
        eps= 1e-15  # Малое значение для предотвращения ошибок логарифма
        y_pred = np.clip(y_pred, eps, 1 - eps)  # Ограничение значений
        loss = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return np.mean(loss)

    def backward(self, y_pred, y, X):
        pass

class Optimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def step(self, model, dw, db):
        model.w -= self.learning_rate * dw
        model.b -= self.learning_rate * db

# Задание

Дедлайн: чт 07.08 18:00 МСК

1. Загрузить датасет, разбить его на два датасета: первый для обучения, второй - для проверки качества (см. функцию [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)) **(1 балл)**
2. Посчитать производные функции потерь по $w$ и по $b$ **(3 балла)**
3. Реализовать цикл обучения, обновляя параметры логистической регрессии. Необходим график зависимости от номера итерации **(5 баллов)**:
$$w = w - \alpha \cdot \frac{\partial L}{\partial w}$$
<!-- <br> -->
$$b = b - \alpha \cdot \frac{\partial L}{\partial b}$$
Обратите внимание на величину $\alpha$. Ее надо подобрать, иначе алгоритм не будет обучаться.

4. Посчитать качество модели по метрике [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) **(1 балл)**

5. (бонус - оценивается только при успешном выполнении предыдущих пунктов) осуществить перебор по $\alpha$ и найти его оптимальное значение (обеспечивающее максимальную метрику accuracy при достаточно небольшом количестве итераций обучения) **(2 балла)**